## Convolutional Neural Networks

In this assignment, we will learn about convolutional neural networks. We will create a CNN and learn to classify image data.

In this lecture, we will use the image data generator to classify our data. The data is loaded below:

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization 
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import tensorflow as tf

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data_dir = '/content/drive/MyDrive/dogs-vs-cats-processed.zip (Unzipped Files)/dogs-vs-cats-processed/train'
validation_data_dir = '/content/drive/MyDrive/dogs-vs-cats-processed.zip (Unzipped Files)/dogs-vs-cats-processed/test'

img_width, img_height = 150, 150
batch_size = 80

In [5]:
#This block of code is used to ensure the input shape is correct

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

Define a train data generator with shear range of 0.3, zoom range of 0.1 and rescale to 1./255 (note that we must make 1 a float to produce a correct fraction). Use the ImageDataGenerator function.

In [6]:
# make training data generator
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=.3,zoom_range=.1)


Define a test data generator that only rescales to 1./255. Use the ImageDataGenerator function.

In [7]:
# make test data generator
test_datagen = ImageDataGenerator(rescale=1. / 255)


The train generator and the test generator are defined below:

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #shuffle=False,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


We'll start with a simple model. In CNNs, we first convolve the to extract features and then we add the dense layers. 

Create a model with one layer of convolution of size 64, one layer of activation, one layer of max pooling with pool size (2,2) and then one flattening layer, one dense layer of unit size 64 with a ReLU activation and one dense output layer. The output layer should have a sigmoid activation.

In [9]:
# make cnn model
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 350464)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                22429760  
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

Compile the model using RMSprop.

In [11]:
# compile RMSprop model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])


Fit the model using a fit generator. Use 50 epochs, 25 training steps and 15 validation steps

In [12]:
# fit model
## had to use validation steps of 5 to prevent running out of validation data
model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=50, 
          validation_data=validation_generator, 
          validation_steps=5)


Epoch 1/50
25/25 [==============================] - 19s 647ms/step - loss: 12.0658 - accuracy: 0.5061 - val_loss: 0.7045 - val_accuracy: 0.5700
Epoch 2/50
25/25 [==============================] - 16s 651ms/step - loss: 0.7210 - accuracy: 0.5529 - val_loss: 0.6780 - val_accuracy: 0.6000
Epoch 3/50
25/25 [==============================] - 16s 653ms/step - loss: 0.6379 - accuracy: 0.6431 - val_loss: 0.6309 - val_accuracy: 0.7000
Epoch 4/50
25/25 [==============================] - 16s 648ms/step - loss: 0.5999 - accuracy: 0.6955 - val_loss: 0.5842 - val_accuracy: 0.7175
Epoch 5/50
25/25 [==============================] - 16s 647ms/step - loss: 0.6430 - accuracy: 0.7260 - val_loss: 0.6166 - val_accuracy: 0.6700
Epoch 6/50
25/25 [==============================] - 16s 648ms/step - loss: 0.5038 - accuracy: 0.7630 - val_loss: 0.5878 - val_accuracy: 0.6825
Epoch 7/50
25/25 [==============================] - 16s 654ms/step - loss: 0.4860 - accuracy: 0.7856 - val_loss: 0.6192 - val_accuracy: 0.692

Create a new model by adding an additional group of convolution, activation and max pooling layers before the flatten layer. Make the convolution layer of unit size 32. Keep everything else the same.

In [13]:
# make cnn model with two conv layers
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))


Fit and compile the model in the same way you did with the previous model. How did the results improve?

In [14]:
# compile RMSprop model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# fit model
## had to use validation steps of 5 to prevent running out of validation data
model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=50, 
          validation_data=validation_generator, 
          validation_steps=5)


Epoch 1/50
25/25 [==============================] - 18s 677ms/step - loss: 1.2096 - accuracy: 0.5317 - val_loss: 0.7231 - val_accuracy: 0.5025
Epoch 2/50
25/25 [==============================] - 16s 658ms/step - loss: 0.6866 - accuracy: 0.5570 - val_loss: 0.7567 - val_accuracy: 0.5075
Epoch 3/50
25/25 [==============================] - 16s 650ms/step - loss: 0.6734 - accuracy: 0.5967 - val_loss: 0.6639 - val_accuracy: 0.6025
Epoch 4/50
25/25 [==============================] - 16s 653ms/step - loss: 0.6651 - accuracy: 0.6380 - val_loss: 0.6680 - val_accuracy: 0.5700
Epoch 5/50
25/25 [==============================] - 16s 653ms/step - loss: 0.6020 - accuracy: 0.6862 - val_loss: 0.6505 - val_accuracy: 0.6375
Epoch 6/50
25/25 [==============================] - 16s 652ms/step - loss: 0.5705 - accuracy: 0.7070 - val_loss: 0.6255 - val_accuracy: 0.6375
Epoch 7/50
25/25 [==============================] - 16s 655ms/step - loss: 0.5299 - accuracy: 0.7393 - val_loss: 0.5981 - val_accuracy: 0.6875

Create a new model based on the model above. Add an additional dense layer of size 64 with a ReLU activation after the flatten layer.

In [15]:
# make cnn model with two conv layers and two dense layers
# instantiate model
model = Sequential()
# convolution layer 1
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# convolution layer 2
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#flatten layer
model.add(Flatten())
# dense layer 1
model.add(Dense(64))
model.add(Activation('relu'))
#dense layer 2
model.add(Dense(64))
model.add(Activation('relu'))
#dense layer 3 / output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))


Fit and compile in the same way as above. Describe the difference in performance and speed.

In [16]:
# compile RMSprop model
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# fit model
## had to use validation steps of 5 to prevent running out of validation data
model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=50, 
          validation_data=validation_generator, 
          validation_steps=5)

Epoch 1/50
25/25 [==============================] - 17s 667ms/step - loss: 1.0933 - accuracy: 0.5169 - val_loss: 0.6841 - val_accuracy: 0.5900
Epoch 2/50
25/25 [==============================] - 16s 655ms/step - loss: 0.6883 - accuracy: 0.5962 - val_loss: 0.6728 - val_accuracy: 0.5500
Epoch 3/50
25/25 [==============================] - 16s 648ms/step - loss: 0.6965 - accuracy: 0.5963 - val_loss: 0.6488 - val_accuracy: 0.6500
Epoch 4/50
25/25 [==============================] - 16s 654ms/step - loss: 0.6626 - accuracy: 0.6449 - val_loss: 0.6465 - val_accuracy: 0.6225
Epoch 5/50
25/25 [==============================] - 16s 652ms/step - loss: 0.6746 - accuracy: 0.6863 - val_loss: 0.6389 - val_accuracy: 0.6550
Epoch 6/50
25/25 [==============================] - 16s 659ms/step - loss: 0.5904 - accuracy: 0.7325 - val_loss: 0.6429 - val_accuracy: 0.6600
Epoch 7/50
25/25 [==============================] - 16s 659ms/step - loss: 0.5275 - accuracy: 0.7310 - val_loss: 0.6623 - val_accuracy: 0.6500

Fit and compile using the Adam optimizer. Describe the difference in performance between the Adam and RMSprop optimizers.

In [17]:
# compile Adam model
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

# fit model
## had to use validation steps of 5 to prevent running out of validation data
model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=50, 
          validation_data=validation_generator, 
          validation_steps=5)

Epoch 1/50
25/25 [==============================] - 17s 664ms/step - loss: 0.0189 - accuracy: 0.9935 - val_loss: 2.5723 - val_accuracy: 0.6475
Epoch 2/50
25/25 [==============================] - 16s 651ms/step - loss: 0.0160 - accuracy: 0.9938 - val_loss: 2.6727 - val_accuracy: 0.6700
Epoch 3/50
25/25 [==============================] - 16s 648ms/step - loss: 0.0125 - accuracy: 0.9968 - val_loss: 2.1494 - val_accuracy: 0.6700
Epoch 4/50
25/25 [==============================] - 16s 659ms/step - loss: 0.0121 - accuracy: 0.9948 - val_loss: 2.2464 - val_accuracy: 0.7075
Epoch 5/50
25/25 [==============================] - 16s 657ms/step - loss: 0.0092 - accuracy: 0.9979 - val_loss: 2.1433 - val_accuracy: 0.6825
Epoch 6/50
25/25 [==============================] - 16s 653ms/step - loss: 0.0241 - accuracy: 0.9934 - val_loss: 1.9916 - val_accuracy: 0.6325
Epoch 7/50
25/25 [==============================] - 16s 648ms/step - loss: 0.0170 - accuracy: 0.9934 - val_loss: 2.0988 - val_accuracy: 0.6675